##### Solution from Shreyas Pobbi

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy.stats import mode
# import modelvis
from sklearn.ensemble import RandomForestClassifier
%matplotlib

Using matplotlib backend: MacOSX


In [110]:
import csv

## Frame 
Given building details, your task is to build a model that can predict the extent of damage that has been done to a building after an earthquake.

## ACQUIRE
train.csv, test.csv, building_ownership_usage.csv, building_structure.csv, sample_submission.csv

In [40]:
df = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/train.csv')
df_bo = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/Building_Ownership_Use.csv')
df_str = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/building_structure.csv')

In [41]:
df.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id'],
      dtype='object')

## REFINE
Predict damage_grade

In [42]:
df.damage_grade.value_counts()

Grade 5    210825
Grade 4    152244
Grade 3    122288
Grade 2     85084
Grade 1     61320
Name: damage_grade, dtype: int64

In [43]:
df.shape

(631761, 14)

In [44]:
df.dtypes

area_assesed                              object
building_id                               object
damage_grade                              object
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object

In [45]:
df.isnull().sum()

area_assesed                                 0
building_id                                  0
damage_grade                                 0
district_id                                  0
has_geotechnical_risk                        0
has_geotechnical_risk_fault_crack            0
has_geotechnical_risk_flood                  0
has_geotechnical_risk_land_settlement        0
has_geotechnical_risk_landslide              0
has_geotechnical_risk_liquefaction           0
has_geotechnical_risk_other                  0
has_geotechnical_risk_rock_fall              0
has_repair_started                       33417
vdcmun_id                                    0
dtype: int64

In [46]:
#Replacing null values by mode(0) value of has_repair_started
df.has_repair_started.fillna(0, inplace=True)

In [47]:
df.isnull().sum()

area_assesed                             0
building_id                              0
damage_grade                             0
district_id                              0
has_geotechnical_risk                    0
has_geotechnical_risk_fault_crack        0
has_geotechnical_risk_flood              0
has_geotechnical_risk_land_settlement    0
has_geotechnical_risk_landslide          0
has_geotechnical_risk_liquefaction       0
has_geotechnical_risk_other              0
has_geotechnical_risk_rock_fall          0
has_repair_started                       0
vdcmun_id                                0
dtype: int64

In [48]:
df.area_assesed.value_counts()

Both                   382992
Building removed       131346
Exterior                99030
Not able to inspect     16575
Interior                 1818
Name: area_assesed, dtype: int64

In [49]:
grade = {'Grade 5': 5, 'Grade 4': 4, 'Grade 3': 3, 'Grade 2': 2, 'Grade 1': 1}
area = {'Both': 0, 'Building removed': 1, 'Exterior': 2, 'Not able to inspect': 3, 'Interior': 4}

In [50]:
df.damage_grade = df.damage_grade.map(grade.get)
df.area_assesed = df.area_assesed.map(area.get)

In [51]:
df.dtypes

area_assesed                               int64
building_id                               object
damage_grade                               int64
district_id                                int64
has_geotechnical_risk                    float64
has_geotechnical_risk_fault_crack          int64
has_geotechnical_risk_flood                int64
has_geotechnical_risk_land_settlement      int64
has_geotechnical_risk_landslide            int64
has_geotechnical_risk_liquefaction         int64
has_geotechnical_risk_other                int64
has_geotechnical_risk_rock_fall            int64
has_repair_started                       float64
vdcmun_id                                  int64
dtype: object

In [52]:
df_bo.columns

Index(['building_id', 'district_id', 'vdcmun_id', 'ward_id',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other'],
      dtype='object')

In [53]:
df_str.columns

Index(['building_id', 'district_id', 'vdcmun_id', 'ward_id',
       'count_floors_pre_eq', 'count_floors_post_eq', 'age_building',
       'plinth_area_sq_ft', 'height_ft_pre_eq', 'height_ft_post_eq',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq'],
      dtype='object')

### Mering with building sturture and ownership data

In [55]:
result = pd.merge(df_str, df_bo, on='building_id')

In [56]:
df=pd.merge(df,result,on="building_id")

In [58]:
land = {'Flat': 0, 'Moderate slope': 1, 'Steep slope': 2}
foundation = {'Mud mortar-Stone/Brick': 0, 'Bamboo/Timber': 1, 'Cement-Stone/Brick': 2, 'RC': 3, 'Other': 4}
roof = {'Bamboo/Timber-Light roof':0, 'Bamboo/Timber-Heavy roof':1, 'RCC/RB/RBC': 3}
grnd_floor = {'Mud':0, 'Brick/Stone':1, 'RC':2, 'Timber':3, 'Other':4}
other_floor = {'TImber/Bamboo-Mud': 0, 'Timber-Planck':1, 'Not applicable':2, 'RCC/RB/RBC':3}
pos = {'Not attached':0, 'Attached-1 side':1, 'Attached-2 side':2, 'Attached-3 side':3}
plan = {'Rectangular':0, 'Square':1, 'L-shape':2, 'Multi-projected':3, 'T-shape':4, 'Others':5, 'U-shape':6, 'Building with Central Courtyard':7, 
        'E-shape':8, 'H-shape':9}
condition = {'Damaged-Not used':0, 'Damaged-Repaired and used':1, 'Damaged-Used in risk':2, 'Damaged-Rubble unclear':3, 'Damaged-Rubble clear':4, 
             'Not damaged':5, 'Damaged-Rubble Clear-New building built':6, 'Covered by landslide':7}
legal = {'Private':0, 'Public':1, 'Institutional': 2, 'Other':3}

In [59]:
df.land_surface_condition = df.land_surface_condition.map(land.get)

In [61]:
df.foundation_type = df.foundation_type.map(foundation.get)

In [63]:
df.roof_type = df.roof_type.map(roof.get)

In [64]:
df.ground_floor_type = df.ground_floor_type.map(grnd_floor.get)

In [66]:
df.other_floor_type = df.other_floor_type.map(other_floor.get)

In [67]:
df.position = df.position.map(pos.get)

In [69]:
df.plan_configuration = df.plan_configuration.map(plan.get)

In [70]:
df.condition_post_eq = df.condition_post_eq.map(condition.get)

In [71]:
df.legal_ownership_status = df.legal_ownership_status.map(legal.get)

In [74]:
df.count_families.fillna(0, inplace=True)

In [76]:
df.columns

Index(['area_assesed', 'building_id', 'damage_grade', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_bri

## MODEL FIT

In [77]:
'''
X = df[['area_assesed', 'district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id']]
'''
X = df[['area_assesed','district_id',
       'has_geotechnical_risk', 'has_geotechnical_risk_fault_crack',
       'has_geotechnical_risk_flood', 'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'condition_post_eq', 'district_id_y', 'vdcmun_id_y', 'ward_id_y',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other']]

y = df.damage_grade

In [78]:
model = RandomForestClassifier(max_depth=19, n_estimators=100)
model.fit(X, y)
# score = cross_val_score(model, X, y, scoring="accuracy", cv=5, n_jobs=-1)
# print(np.mean(score))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=19, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
param = {'max_depth': np.arange(2,20), 'n_estimators': [50, 70, 100]}
clf = GridSearchCV(model, param, scoring='accuracy', return_train_score=True)
clf.fit(X, y)
clf.best_params_

#got output max_depth=19, n_estimators=100

## TESTING

#### test.csv

In [95]:
df_test = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/test.csv')
df_test=pd.merge(df_test,result,on="building_id")

In [70]:
df_test.columns

Index(['area_assesed', 'building_id', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id'],
      dtype='object')

In [96]:
#after merging
df_test.columns

Index(['area_assesed', 'building_id', 'district_id', 'has_geotechnical_risk',
       'has_geotechnical_risk_fault_crack', 'has_geotechnical_risk_flood',
       'has_geotechnical_risk_land_settlement',
       'has_geotechnical_risk_landslide', 'has_geotechnical_risk_liquefaction',
       'has_geotechnical_risk_other', 'has_geotechnical_risk_rock_fall',
       'has_repair_started', 'vdcmun_id', 'district_id_x', 'vdcmun_id_x',
       'ward_id_x', 'count_floors_pre_eq', 'count_floors_post_eq',
       'age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq',
       'height_ft_post_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_supers

In [100]:
df_test.land_surface_condition = df_test.land_surface_condition.map(land.get)
df_test.foundation_type = df_test.foundation_type.map(foundation.get)
df_test.roof_type = df_test.roof_type.map(roof.get)
df_test.ground_floor_type = df_test.ground_floor_type.map(grnd_floor.get)
df_test.other_floor_type = df_test.other_floor_type.map(other_floor.get)
df_test.position = df_test.position.map(pos.get)
df_test.plan_configuration = df_test.plan_configuration.map(plan.get)

In [102]:
df_test.condition_post_eq = df_test.condition_post_eq.map(condition.get)
df_test.legal_ownership_status = df_test.legal_ownership_status.map(legal.get)

In [106]:
df_test.has_repair_started.fillna(0, inplace=True)

In [98]:
area = {'Both': 0, 'Building removed': 1, 'Exterior': 2, 'Not able to inspect': 3, 'Interior': 4}
df_test.area_assesed = df_test.area_assesed.map(area.get)

In [108]:
df_test.dtypes

area_assesed                                int64
building_id                                object
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
district_id_x                               int64
vdcmun_id_x                                 int64
ward_id_x                                   int64
count_floors_pre_eq                         int64
count_floors_post_eq                        int64
age_building                                int64
plinth_area_sq_ft                           int64


In [74]:
df_test.head(5)

,area_assesed,building_id,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,has_repair_started,vdcmun_id
0,0,a3380c4f75,7,0.0,0,0,0,0,0,0,0,1.0,701
1,0,a338a4e653,7,0.0,0,0,0,0,0,0,0,1.0,701
2,1,a338a4e6b7,7,0.0,0,0,0,0,0,0,0,1.0,701
3,0,a33a6eaa3a,7,0.0,0,0,0,0,0,0,0,1.0,701
4,1,a33b073ff6,7,0.0,0,0,0,0,0,0,0,1.0,701


In [114]:
#df_test.building_id = df_test.building_id.mask(int(str(df_test.building_id), 16))
#df_test.building_id = np.where(int(df_test.building_id, 16), df_test.building_id)
df_test.building_id = df_test.building_id.apply(lambda x: int(x, 16))

In [115]:
df_test.dtypes

area_assesed                                int64
building_id                                 int64
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
district_id_x                               int64
vdcmun_id_x                                 int64
ward_id_x                                   int64
count_floors_pre_eq                         int64
count_floors_post_eq                        int64
age_building                                int64
plinth_area_sq_ft                           int64


In [117]:
df_test_cp = df_test.copy()
df_test_cp.drop('building_id', axis=1, inplace=True)
df_test_cp.head(3)

,area_assesed,district_id,has_geotechnical_risk,has_geotechnical_risk_fault_crack,has_geotechnical_risk_flood,has_geotechnical_risk_land_settlement,has_geotechnical_risk_landslide,has_geotechnical_risk_liquefaction,has_geotechnical_risk_other,has_geotechnical_risk_rock_fall,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,0,7,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,7,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,7,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# int('a3380c4f75', 16)
# hex(701020000117)

In [118]:
# preValue = model.predict(df_test_cp.head(9999))
preValue = model.predict([df_test_cp.iloc[400000]])

In [119]:
preValue

array([4])

In [107]:
with open('output.csv', 'w') as csvfp:
    wr = csv.writer(csvfp)
    for i in range(1, len(preValue)):
        wr.writerow([hex(df_test.building_id[i]), 'Grade '+str(preValue[i])])
        
csvfp.close()

In [108]:
df1 = df_test_cp[10000:]

In [141]:
df_test_cp.isnull().sum()

area_assesed                                 0
district_id                                  0
has_geotechnical_risk                        0
has_geotechnical_risk_fault_crack            0
has_geotechnical_risk_flood                  0
has_geotechnical_risk_land_settlement        0
has_geotechnical_risk_landslide              0
has_geotechnical_risk_liquefaction           0
has_geotechnical_risk_other                  0
has_geotechnical_risk_rock_fall              0
has_repair_started                       21922
vdcmun_id                                    0
dtype: int64

In [142]:
df_test_cp.has_repair_started.fillna(0, inplace=True)

In [143]:
df_test_cp.isnull().sum()

area_assesed                             0
district_id                              0
has_geotechnical_risk                    0
has_geotechnical_risk_fault_crack        0
has_geotechnical_risk_flood              0
has_geotechnical_risk_land_settlement    0
has_geotechnical_risk_landslide          0
has_geotechnical_risk_liquefaction       0
has_geotechnical_risk_other              0
has_geotechnical_risk_rock_fall          0
has_repair_started                       0
vdcmun_id                                0
dtype: int64

In [118]:
try:
    with open('output_merge.csv', 'w') as csvfp:
        wr = csv.writer(csvfp)
        for i in range(len(df_test_cp)):
    #         print([format(df_test.building_id[i]), 'Grade '+str(model.predict([df_test_cp.iloc[i]])[0])])
            wr.writerow([format(df_test.building_id[i], 'x'), 'Grade '+str(model.predict([df_test_cp.iloc[i]])[0])])
    #         break
except Exception as ex:
    print(ex)
finally:
    csvfp.close()

#### Accuracy without merge and only RandomForestClassifier is 0.61545

#### Accuracy with merge and RandomForestClassifier is 0.73827